# Using Quandl Free Euronext Market Data

In [1]:
import quasardb
import quasardb.pandas as qdbpd
import pandas as pd
import datetime as dt
import numpy as np
import quandl as qd
import os
import urllib.request
from tqdm import trange, tqdm_notebook
print(quasardb.version())
import plotly.graph_objects as go


3.5.0master


In [2]:
#You need a Quandl API key to run this sample
#It is free just register on quandl.com
qd.ApiConfig.api_key = os.environ['QUANDL']

In [3]:
#get list of stocks on Euronext on Quandl
url='https://www.quandl.com/api/v3/databases/EURONEXT/metadata?api_key='+qd.ApiConfig.api_key
urllib.request.urlretrieve(url, 'enext.zip')
enext=pd.read_csv('enext.zip')
#make sure dates are in datetime64[ns] format
enext.to_date = enext.to_date.astype('datetime64[ns]')
#filter only active stocks
activeEnext=enext[enext['to_date']==max(enext['to_date'])]

In [4]:
#number of active stocks
activeEnext.shape

(935, 6)

In [5]:
#connect to QDB server
c = quasardb.Cluster("qdb://qdb-server:2836")

In [6]:
#Query all stocks from Quandl and insert them in QDB using TAG EURONEXT
for i in tqdm_notebook(range(len(activeEnext.index)), desc='looping through stocks'):
    data=qd.get('EURONEXT/'+activeEnext.iloc[i].code)
    qdbpd.write_dataframe(data, c, activeEnext.iloc[i].code,create=True)
    t = c.ts(activeEnext.iloc[i].code)
    t.attach_tag('EURONEXT')

In [6]:
#a bit of code to format querys as pandas dataframes we will use after
def return_as_pandas(query_result):
    final_result=pd.DataFrame()
    for table in query_result.tables:
        tmp = dict()
        for col in query_result.tables[table]:
            tmp[col.name] = col.data
        result_as_pandas = pd.DataFrame.from_dict(tmp, orient='columns').set_index('$timestamp')
        result_as_pandas['table']=table
        final_result =  pd.concat([final_result,result_as_pandas])
    return final_result

In [18]:
#Let's query prices on one stock, Renault - Euronext ID = RNO
stock='XIL'
pandas_result =return_as_pandas (c.query("select Last FROM "+stock)
                                 .run()
)


In [19]:
#plotting the stock data
fig = go.Figure([go.Scatter(x=pandas_result.index, y=pandas_result.Last,name=stock)])
fig.update_layout(title_text='RNO Historical Price',
                  xaxis_rangeslider_visible=True)
fig.show()

In [26]:
#using TAG EURONEXT to manipulate data
#here return all values for 1 day
pandas_result =return_as_pandas (
    c.query("select * FROM FIND ( tag ='EURONEXT' ) in range ( 2019-09-21, -1d)")
    .run()
)
pandas_result[:5]

,Open,High,Low,Last,Volume,Turnover,table
$timestamp,,,,,,,
2019-09-20,35.30,35.30,34.850,35.300,1571.0,55206.0,XIL
2019-09-20,79.60,80.00,79.400,80.000,11277.0,901522.0,WEHB
2019-09-20,57.00,57.00,57.000,57.000,105.0,5985.0,WEB
2019-09-20,25.05,25.15,24.300,24.700,8891.0,218036.0,WAVE
2019-09-20,2.09,2.09,1.975,1.975,23425.0,46886.0,VTX


In [24]:
#Return all stocks having a turnover > 400M between 09-11 and 09-21 
pandas_result =return_as_pandas (
    c.query("select * FROM FIND ( tag ='EURONEXT' ) in range ( 2019-09-21, -10d) where Turnover > 400000000")
    .run()
)
pandas_result

Error: Invalid reply from the remote host.